* Represent Sudoku problem as an exact cover problem.
* Use Algorithm X to solve the exact cover problem as NP-complete.
* The implemented technique is a variation of Dancing Links.
* It uses dictionnaries instead of doubly-linked lists to represent the matrix of the problem.
* One dictionary containing all the constraints, and one mapping dictionary to map cells to their constraints.

1. Constract 'Constraint Dictionary' and 'Mapping Dictionary'
2. Removing constraints for given clues.
3. Use Algorithm X to satisfy the constraints for each empty cell, and then remove constraints for these cells from 'Constraint Dictionary'. Using recursion and backtraking.
4. Once, the 'Constraint Dictionary' is empty, it means that all constraints are satisfied, so we have a solution.

In [47]:
# Import the necessary modules

from itertools import product
import urllib.request
import time

In [48]:
# Read the Sudoku grid from the provided source
def readSudoku(url):

  # Get the whole content of the source
  with urllib.request.urlopen(url) as response:
    content = response.read().decode("utf-8")

  # Filter out rows (a line is a row)
  rows = [line for line in content.split('\r\n')]

  # Convert each row in a list of integers
  # Getting all the elemets on Sudoku grid
  grid = [[],[],[],[],[],[],[],[],[]]
  for row in range(0,9):
    grid[row] = [int(x) for x in str(rows[row])]

  return grid

In [49]:
# This function provides with a meaningful representation of the Sudoku grid
# The all grid is divided in 9 sub-grids

def print_grid(grid):
  for row in range(9):
    if row % 3 == 0 and row != 0:
      print("- - - - - - - - - - - -")
    for col in range(9):
      if col % 3 == 0 and col != 0:
        print(" | ", end="")
      if col == 8:
        print(grid[row][col])
      else:
        print(f'{str(grid[row][col])} ', end="")

In [50]:
def solve_sudoku(size, grid):
    global start_time
    start_time = time.time()

    R, C = size
    N = R * C
#-----------------------------------------------------------------------------------------
    # 1. Constract 'Constraint Dictionary' and 'Mapping Dictionary'

    # Create a list representing all constraints in the Sudoku problem.
    # Each constrain is a tuple of two elements:
    #       1. The type of constrain (string)
    #       2. The corresponding indices or values involved in the constrain (tuple)
    X = ([("rc", rc) for rc in product(range(N), range(N))] +
         [("rn", rn) for rn in product(range(N), range(1, N + 1))] +
         [("cn", cn) for cn in product(range(N), range(1, N + 1))] +
         [("bn", bn) for bn in product(range(N), range(1, N + 1))])

    # Y (Mapping Dictionary)---> Mapping cells to constraints:
    #              For each combination row, column(cell) and number,
    #              Y contains its corresponding list of constraints.
    #              Key: The combination (r,c,n) as tuple
    #              Value: List of constraints
    Y = dict()
    for r, c, n in product(range(N), range(N), range(1, N + 1)):
        b = (r // R) * R + (c // C) # Box number
        Y[(r, c, n)] = [
            ("rc", (r, c)),
            ("rn", (r, n)),
            ("cn", (c, n)),
            ("bn", (b, n))]

    # Represent the matrix of the excat cover problem
    # X---> Constraint Dictionary
    # Y---> Mapping Dictionary
    X, Y = exact_cover(X, Y)
#------------------------------------------------------------------------------
    # 2. Removing constraints for given clues.

    for i, row in enumerate(grid):
        for j, n in enumerate(row):
            if n:
                select(X, Y, (i, j, n))
#----------------------------------------------------------------------------
    # 3,4. Perform Algorithm X until you have a solution
    for solution in solve(X,Y,[]) :
        for (r, c, n) in solution:
            grid[r][c] = n
        yield grid

In [51]:
# From constrain list (X) and from Mapping Dictionary (Y),
# create the Constraint Dictionary --> X (which maps constraints to cells).

def exact_cover(X, Y):
    X = {j: set() for j in X}
    for i, row in Y.items():
        for j in row:
            X[j].add(i)
    return X, Y

In [52]:
# Implementation of Algorithm X

def solve(X, Y, solution):
    global caller
    global start_time
    caller +=1

    # 0) Matrix has no columns, partial solution is valid; terminate successfully
    if not X:
        end_time = time.time()
        yield list(solution)
        print(f'\n{end_time-start_time}\n')
        print(f'\n{caller}\n')

    # 1) Otherwise choose a column c
    else:
        c = min(X, key=lambda c: len(X[c])) # use heuristic (column with the fewest 1s )

        # 2) Choose a row r such that A[r][c] = 1
        for r in list(X[c]):

            # 3) Include row r in the partial solution
            solution.append(r)

            # 4) Reduce matrix A
            cols = select(X, Y, r)

            # 5) Repeat this algorithm recursively on the reduced matrix A.
            for s in solve(X, Y, solution):
                yield s

            # Reverse changes made by "select(X, Y, r)",
            # in order to look for other solutions.
            deselect(X, Y, r, cols)
            solution.pop()



# Reduce matrix A
def select(X, Y, r):
    cols = []
    for j in Y[r]:
        for i in X[j]:
            for k in Y[i]:
                if k != j:
                    X[k].remove(i)
        cols.append(X.pop(j))
    return cols

# Reverse changes made by "select(X, Y, r)", in order to look for other solutions
def deselect(X, Y, r, cols):
    for j in reversed(Y[r]):
        X[j] = cols.pop()
        for i in X[j]:
            for k in Y[i]:
                if k != j:
                    X[k].add(i)

In [24]:
grid = [
    [8,0,0,0,0,0,0,0,0],
    [0,0,3,6,0,0,0,0,0],
    [0,7,0,0,9,0,2,0,0],
    [0,5,0,0,0,7,0,0,0],
    [0,0,0,0,4,5,7,0,0],
    [0,0,0,1,0,0,0,3,0],
    [0,0,1,0,0,0,0,6,8],
    [0,0,8,5,0,0,0,1,0],
    [0,9,0,0,0,0,4,0,0]
]

In [17]:
grid = [
    [1,0,0,0,4,0,0,9,0],
    [0,0,7,0,6,0,0,0,0],
    [0,8,0,7,0,9,3,0,0],
    [0,0,2,3,0,7,0,1,0],
    [0,0,0,6,0,0,0,0,0],
    [0,4,0,0,0,0,0,0,2],
    [5,0,0,0,0,0,8,0,0],
    [0,0,4,2,0,1,0,3,0],
    [0,0,0,0,9,0,0,0,0]
]

In [53]:
grid = readSudoku("https://uchida.cis.k.hosei.ac.jp/2015/optBprob-hard2.txt")

In [13]:
grid[0][5] = 0

In [54]:
print_grid(grid)

0 0 0  | 0 0 1  | 0 3 0
0 9 5  | 6 0 0  | 0 0 0
0 8 0  | 0 0 0  | 0 0 0
- - - - - - - - - - - -
3 0 0  | 0 9 0  | 0 0 0
0 0 0  | 8 0 0  | 0 0 0
4 0 0  | 0 0 0  | 6 2 0
- - - - - - - - - - - -
0 0 0  | 0 0 0  | 0 0 7
0 0 0  | 0 0 0  | 5 0 8
2 0 0  | 0 0 4  | 0 0 0


In [55]:
start_time = time.time()
caller = 0
for solutions in solve_sudoku((3, 3), grid):
  print_grid(solutions)
  print('\n\n')

7 2 4  | 9 5 1  | 8 3 6
1 9 5  | 6 3 8  | 4 7 2
6 8 3  | 2 4 7  | 9 5 1
- - - - - - - - - - - -
3 1 2  | 4 9 6  | 7 8 5
5 6 9  | 8 7 2  | 3 1 4
4 7 8  | 3 1 5  | 6 2 9
- - - - - - - - - - - -
8 3 1  | 5 6 9  | 2 4 7
9 4 7  | 1 2 3  | 5 6 8
2 5 6  | 7 8 4  | 1 9 3




0.0073888301849365234


65



In [37]:
urls =['https://uchida.cis.k.hosei.ac.jp/2015/optBprob-easy.txt','https://uchida.cis.k.hosei.ac.jp/2015/optBprob-medium.txt',
       'https://uchida.cis.k.hosei.ac.jp/2015/optBprob-hard1.txt','https://uchida.cis.k.hosei.ac.jp/2015/optBprob-hard2.txt',
       'https://uchida.cis.k.hosei.ac.jp/2015/optBprob-hard3.txt','https://uchida.cis.k.hosei.ac.jp/2015/hard4.txt','https://uchida.cis.k.hosei.ac.jp/2015/hard5.txt']

for url in urls:
  grid = readSudoku(url)
  print('----------------------------------------------')
  print_grid(grid)
  print("\n")

  start_time = time.time()
  caller = 0
  for solutions in solve_sudoku((3, 3), grid):
    print_grid(solutions)

----------------------------------------------
9 2 1  | 7 3 5  | 4 8 6
4 7 6  | 2 8 9  | 5 1 3
5 8 3  | 1 6 4  | 2 7 9
- - - - - - - - - - - -
0 0 0  | 6 5 2  | 0 0 0
0 0 0  | 8 4 3  | 0 0 0
0 0 0  | 9 1 7  | 0 0 0
- - - - - - - - - - - -
2 4 7  | 5 9 8  | 3 6 1
6 5 9  | 3 2 1  | 7 4 8
1 3 8  | 4 7 6  | 9 2 5


9 2 1  | 7 3 5  | 4 8 6
4 7 6  | 2 8 9  | 5 1 3
5 8 3  | 1 6 4  | 2 7 9
- - - - - - - - - - - -
8 9 4  | 6 5 2  | 1 3 7
7 1 5  | 8 4 3  | 6 9 2
3 6 2  | 9 1 7  | 8 5 4
- - - - - - - - - - - -
2 4 7  | 5 9 8  | 3 6 1
6 5 9  | 3 2 1  | 7 4 8
1 3 8  | 4 7 6  | 9 2 5

0.003282308578491211


19

----------------------------------------------
5 0 0  | 3 6 0  | 0 0 1
2 0 4  | 0 0 0  | 0 3 0
0 7 0  | 0 9 0  | 0 0 4
- - - - - - - - - - - -
0 2 0  | 0 0 0  | 0 9 0
0 0 0  | 4 0 8  | 0 0 0
0 3 0  | 0 0 0  | 0 8 0
- - - - - - - - - - - -
3 0 0  | 0 8 0  | 0 1 0
0 8 0  | 0 0 0  | 7 0 6
7 0 0  | 0 4 5  | 0 0 8


5 9 8  | 3 6 4  | 2 7 1
2 1 4  | 8 5 7  | 6 3 9
6 7 3  | 2 9 1  | 8 5 4
- - - - - 